## (30%) Finding similiarity
Distance based metrics are commonly used methods to create similiarity between two items. This similarity will be applied in content-based recommendation. They also have been used for a long time in Information Retrieval methods.

Here is a good article on [Distance Metrics for Fun and Profit](http://www.benfrederickson.com/distance-metrics/) by Ben on better understanding of these metrics

Could you based on the provided dataset (stories.csv) to generate similarity matrix between each story? The i, j, cell of this matrix will store the similarity value between story i and story j. You can define your own similarity measures.   

#### Solutions

In [1]:
import pandas as pd
df = pd.read_csv('C:/Akshata/Courses/summer19/large_scale_analytics/stories.csv')
df.head()

,story,title,score,user
0,14274033,"Uber is valued at $70B, you can get it at $999",503,appoets
1,14107522,Jeff Bezos? Annual Letter,547,djyaz1200
2,13647190,India has banned disposable plastic in Delhi,671,SimplyUseless
3,13309610,Startup Puts Everything You Need for a Two-Acr...,511,kungfudoi
4,13860890,The Uber Bombshell About to Drop,1089,dantiberian


## Creating More features from Title

In [2]:
! pip install spacy

In [3]:
! python -m spacy download en_core_web_sm

[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [4]:
import spacy
nlp = spacy.load('en_core_web_sm')
story_id = nlp(df.title[0])
story_title = nlp(df.title[1])

In [5]:
%%time

simi = []

for i in df.title[:50]:
    for j in df.title[:50]:
        story_sim = nlp(i).similarity(nlp(j))
        simi.append([i, j, story_sim])
        
simi = pd.DataFrame(simi, columns = ["story1", "story2", "similarity"])

C:\Users\wel come\Anaconda3\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
C:\Users\wel come\Anaconda3\lib\runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of th

Wall time: 3min 2s


In [18]:
matrix = pd.pivot(simi.story1, simi.story2, simi.similarity)
matrix.head()

story2,1Password Travel Mode: Protect your data when crossing borders,94-year-old Lithium-Ion Battery Inventor Introduces Solid State Battery,?Startup? asks internship applicant to build their app before phone screen,?The moon blew up without warning and for no apparent reason?,A crashed advertisement reveals logs of a facial recognition system,A lawsuit over Costco golf balls,A rising sentiment that IBM?s Watson can?t deliver on its promises,AMA: NY AG Schneiderman on net neutrality and protecting our voice in government,Accidentally destroyed production database on first day of a job,Afraid of Makefiles? Don't be,...,"Welcome, ACLU",What Is Ethereum?,What the CIA WikiLeaks Dump Tells Us: Encryption Works,Whistleblower uncovers London police hacking of journalists and protestors,Why F.E.A.R.?s AI is still the best in first-person shooters,Why Should I Start a Startup?,Why Slack is inappropriate for open source communications,Why We Must Fight for the Right to Repair Our Electronics,Why do many math books have so much detail and so little enlightenment? (2010),YC will hold interviews in Vancouver for founders who can?t get US visas
story1,,,,,,,,,,,,,,,,,,,,,
1Password Travel Mode: Protect your data when crossing borders,1.000000,0.615669,0.526267,0.282099,0.452962,0.603753,0.568304,0.663224,0.422880,0.401323,...,0.349826,0.288586,0.647402,0.606815,0.481515,0.525660,0.537459,0.509209,0.423268,0.533180
94-year-old Lithium-Ion Battery Inventor Introduces Solid State Battery,0.615669,1.000000,0.231294,0.283840,0.307012,0.545492,0.386210,0.640681,0.345800,0.270838,...,0.361119,0.217021,0.624808,0.441978,0.374290,0.340811,0.388952,0.438548,0.188818,0.367617
?Startup? asks internship applicant to build their app before phone screen,0.526267,0.231294,1.000000,0.663362,0.595468,0.513366,0.664473,0.599533,0.723266,0.446101,...,0.291643,0.350131,0.444095,0.375469,0.546044,0.483524,0.588525,0.493852,0.657745,0.499230
?The moon blew up without warning and for no apparent reason?,0.282099,0.283840,0.663362,1.000000,0.620172,0.493994,0.460750,0.599025,0.719966,0.208914,...,0.258612,0.325860,0.346406,0.286706,0.475740,0.373692,0.436561,0.419300,0.528632,0.269890
A crashed advertisement reveals logs of a facial recognition system,0.452962,0.307012,0.595468,0.620172,1.000000,0.639534,0.715270,0.599737,0.757002,0.091097,...,0.094465,0.087147,0.378254,0.574337,0.609272,0.134910,0.584835,0.187485,0.450068,0.394573


## (40%) Model - Matrix Completion by Alternating Least Square (ALS)

Since in practical, customers may refuse to rate some items, study following materials. 

For a better and visual understanding of Matrix Factorization Techniques, read the following links
- The original paper on [Collaborative Filtering for Implicit Feedback Datasets](http://yifanhu.net/PUB/cf.pdf)
- [Finding Similar Music using Matrix Factorization](http://www.benfrederickson.com/matrix-factorization/) by Ben Frederickson
- [Intro to Implicit Matrix Factorization: Classic ALS with Sketchfab Models](http://blog.ethanrosenthal.com/2016/10/19/implicit-mf-part-1/)

We will apply ALS method to fill those missing values (put as 0) for following matrix, a rating matrix for six customers (rows number, say 1,2,3,4,5,6) to seven items (columns number, say A,B,C,D,E,F,G):

R = np.array([[5, 0, 5, 0, 0, 1, 2], [0, 4, 0, 0, 2, 2, 1], [1, 0, 0, 1, 0, 1, 0], [0, 0, 1, 0, 1, 1, 0], [0, 2, 1, 0, 0, 1, 0], [0, 0, 3, 0, 1, 2, 0]])

**Decomposition of the rating matrix**  
R -> m users, n items  
Row in P -> user's affinity to the features (m users, f latent factors/features)  
Row in Q -> item's relation to the features (n items, f latent factors/features)  

- P indicates how much user likes f latent factors  
- Q indicates how much one item obtains f latent factors
- The dot product indicates how much user likes item  
- The decomposition automatically ranks features by their impact on the ratings
- Features may not be intuitive though !
- Model has hyperparameters (regularization, learning rate)

Your task is to write a function using iterative method, e.g., stochastic gradient descent, to factorize
matrix as P with dimension (6 x K) and Q with dimension (K x 7), where K is the number of latent features. 
After you get two factorized matrix P and Q, try to multiply them back, could you recommend the top three items of second customers based on this new rating matrix rating scores (0 value cells will have rating scores now). 


#### Solutions

In [7]:
import numpy as np

def ALS(R):
    
    factors=2
    iterations=5000
    a=0.0002
    b=0.02
    n = len(R)
    m = len(R[0])
    P = np.random.rand(n,factors)
    Q = np.random.rand(m,factors)
    Q = Q.T
    
    for step in range(iterations):
        for i in range (n):
            for j in range(m):
                if R[i][j] > 0:
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])
                    for k in range(factors):
                        P[i][k] = P[i][k] + a * (2 * eij * Q[k][j] - b * P[i][k])
                        Q[k][j] = Q[k][j] + a * (2 * eij * P[i][k] - b * Q[k][j])
        eR = np.dot(P,Q)
        e = 0
        for i in range(n):
            for j in range(m):
                if R[i][j] > 0:
                    e = e + (R[i][j] - np.dot(P[i,:],Q[:,j])**2)
                    for k in range(factors):
                        e = e + (b/2) * ((P[i][k]**2) + (Q[k][j]**2))
        if e < 0.001:
            break
    return P, Q.T

In [8]:
R = np.array([[5, 0, 5, 0, 0, 1, 2], 
              [0, 4, 0, 0, 2, 2, 1], 
              [1, 0, 0, 1, 0, 1, 0], 
              [0, 0, 1, 0, 1, 1, 0],
              [0, 2, 1, 0, 0, 1, 0],
              [0, 0, 3, 0, 1, 2, 0]])

nP, nQ = ALS(R)
nP.dot(nQ.T)

array([[1.68066564, 1.71489243, 1.95190687, 0.87966215, 1.15210204,
        1.55542162, 1.65867002],
       [1.52612254, 1.5814279 , 1.71064105, 0.77331544, 0.9932872 ,
        1.34269023, 1.53315995],
       [1.01994437, 0.99974231, 1.28904305, 0.57689837, 0.78860309,
        1.06182986, 0.96091302],
       [1.17490071, 1.17243627, 1.43182054, 0.64267865, 0.86299982,
        1.16328219, 1.13009942],
       [1.260558  , 1.32869021, 1.35571384, 0.61515559, 0.77144212,
        1.04444792, 1.29140207],
       [1.70048003, 1.73159169, 1.98389245, 0.89373076, 1.17336467,
        1.58388368, 1.67430198]])

In [9]:
# the top three items of second customers based on this new rating matrix rating scores are 1.71, 1.58, 1.53

## (20%) Learning to Rank

**Bayesian Personalized Ranking pairwise loss:**

Maximises the prediction difference between a positive example and a randomly chosen negative example. Useful when only positive interactions are present and optimising ROC AUC is desired.

More details can be found for BPR at 

https://www.coursera.org/lecture/matrix-factorization/personalized-ranking-with-daniel-kluver-s3XJo

https://www.slideshare.net/zenogantner/bayesian-personalized-ranking-for-nonuniformly-sampled-items

**Weighted Approximate-Rank Pairwise loss**:  
Maximises the rank of positive examples by repeatedly sampling negative examples until rank violating one is found. Useful when only positive interactions are present and optimising the top of the recommendation list (precision@k) is desired.WARP deals with (user, positive item, negative item) triplets. 

See following for WARP

https://medium.com/@gabrieltseng/intro-to-warp-loss-automatic-differentiation-and-pytorch-b6aa5083187a


This procedure yields roughly the following algorithm:

- For a given (user, positive item pair), sample a negative item at random from all the remaining items. Compute predictions for both items; if the negative item’s prediction exceeds that of the positive item plus a margin, perform a gradient update to rank the positive item higher and the negative item lower. If there is no rank violation, continue sampling negative items until a violation is found.

- If you found a violating negative example at the first try, make a large gradient update: this indicates that a lot of negative items are ranked higher than positives items given the current state of the model, and the model must be updated by a large amount. If it took a lot of sampling to find a violating example, perform a small update: the model is likely close to the optimum and should be updated at a low rate.

You can install lightfm package and use its fetch_movielens data sets.

In [ ]:
! pip install lightfm

In [2]:
#Get the data

import numpy as np
from lightfm.datasets import fetch_movielens
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

**Movie lens dataset:**

![](img/movielens.png)
GroupLens Research has collected and made available rating data sets from the MovieLens web site (http://movielens.org). The data sets were collected over various periods of time, depending on the size of the set.

http://grouplens.org/datasets/movielens/

**Fetch movielens 100k dataset**

The dataset contains 100,000 interactions from 1000 users on 1700 movies, and is exhaustively described
#in its README http://files.grouplens.org/datasets/movielens/ml-100k-README.txt

This data set consists of:
- 100,000 ratings (1-5) from 943 users on 1682 movies. 
- Each user has rated at least 20 movies. 
- Simple demographic info for the users (age, gender, occupation, zip)

### Questions 1: 

Using lightfm package builtin evaluation methods, e.g., precision_at_k and auc_score
to evaluate recommendation performance from Bayesian Personalized Ranking pairwise loss, and Weighted Approximate-Rank Pairwise loss.  

### Questions 2: 

Can you build a recommendation system based on Weighted Approximate-Rank Pairwise loss and perform recommendation for used with id [3, 25, 450]? You should show both known preference of used and predictions results (recommendatdions).
        

#### Solutions

In [4]:
movielens = fetch_movielens()
training = movielens['train']
testing = movielens['test']

In [12]:
#BPR Model

BPR_model = LightFM(learning_rate=0.05, loss='bpr')
BPR_model.fit(training, epochs=10)
train_precision = precision_at_k(BPR_model, training, k=10).mean()
test_precision = precision_at_k(BPR_model, testing, k=10).mean()
train_auc = auc_score(BPR_model, training).mean()
test_auc = auc_score(BPR_model, testing).mean()
print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.42, test 0.06.
AUC: train 0.84, test 0.82.


In [17]:
#WARP Model

WARP_model = LightFM(learning_rate=0.05, loss='warp')
WARP_model.fit_partial(training, epochs=10)
train_precision = precision_at_k(WARP_model, training, k=10).mean()
test_precision = precision_at_k(WARP_model, testing, k=10).mean()
train_auc = auc_score(WARP_model, training).mean()
test_auc = auc_score(WARP_model, testing).mean()
print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.60, test 0.11.
AUC: train 0.93, test 0.90.


In [22]:
#Recommendation System

data = fetch_movielens(min_rating=5.0)
print(repr(data['train']))
print(repr(data['test']))
model = LightFM(loss='warp')
%time model.fit(data['train'], epochs=30, num_threads=2)
print("Train precision: %.2f" % precision_at_k(model, data['train'], k=5).mean())
print("Test precision: %.2f" % precision_at_k(model, data['test'], k=5).mean())

Train precision: 0.40
Test precision: 0.05


In [24]:
def rec_sys(model, data, user_ids):
    
    users, items = data['train'].shape
    for i in user_ids:
        known_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]
        scores = model.predict(i, np.arange(items))
        top_items = data['item_labels'][np.argsort(scores)]
        print("User %s" % i)
        
        print("    Known positives:")
        for x in known_positives[:3]:
            print("        %s" % x)
​
        print("    Recommended:")
        for x in top_items[:3]:
            print("        %s" % x)

rec_sys(model, data, [3, 25, 450]) 


User 3
     Known positives:
        Contact (1997)
        Air Force One (1997)
        In & Out (1997)
     Recommended:
        Kiss the Girls (1997)
        G.I. Jane (1997)
        Lost Highway (1997)
User 25
     Known positives:
        Fargo (1996)
        Godfather, The (1972)
        L.A. Confidential (1997)
     Recommended:
        Fargo (1996)
        L.A. Confidential (1997)
        Star Wars (1977)
User 450
     Known positives:
        Event Horizon (1997)
        Scream (1996)
        Conspiracy Theory (1997)
     Recommended:
        Scream (1996)
        Game, The (1997)
        Air Force One (1997)


## (10%) Can you implement Discounted cumulative gain (DCG) and Normalized DCG. Then use example of ranking list to demonstrate your implementation.

#### Solutions

In [44]:
import math
import numpy as np

def DCG(ranking_list):
    score = 0.0
    for order, rank in enumerate(ranking_list):
        score += float(rank)/np.log2((order+2))
    return score

def NDCG(r, h):
    return DCG(h)/DCG(r)

print (DCG(h))
print (DCG(r))

6.861126688593502
8.740262365546284


In [45]:
r = [3,3,3,2,2,2] # Ideal case
h = [3,2,3,0,1,2]
print(NDCG(r, h))

0.785002371969948
